In [ ]:
left  = None
right = None
config = None
ref_dem = None
max_disp = 500
dem_gsd = 1.0
img_gsd = 0.25
alignment_method = 'rigid'
output_path = None

# Stereo Config file contents:

In [ ]:
!cat {config}

# Setup Steps

In [2]:
from IPython.display import Image
from asap_stereo import asap

In [ ]:
default_output_dir = '~/auto_asap/hirise/'
left, right = asap.HiRISE().get_hirise_order(left, right)
if output_path == None:
    output_path = default_output_dir + f'a_{left}_{right}'

In [ ]:
!mkdir -p {output_path}

In [ ]:
%cd {output_path}

# Download images (step 1)

In [ ]:
!asap hirise step-one {left} {right} 2>&1 | tee -i -a ./1_download.log ./full_log.log

# Meta Data init for Scripts (step 2)

In [ ]:
!asap hirise step-two

# Hiedr2mosaic (step 3)

In [ ]:
!asap hirise step-three 2>&1 | tee -i -a ./3_mosaic.log ./full_log.log

# Move Files around (Step 4)

In [ ]:
!asap hirise step-four 

# Cam2map4Stereo (Step 5)

In [ ]:
!asap hirise step-five 2>&1 | tee -i -a ./5_cam2map.log ./full_log.log

# Bundle Adjust (Step 6)

In [ ]:
!asap hirise step-six --max-iterations 50 2>&1 | tee -i -a ./6_bundle_adjust.log ./full_log.log

# Parallel Stereo p1 (Step 7)

In [ ]:
!asap hirise step-seven {config} 2>&1 | tee -i -a ./7_parellel_stereo_1.log ./full_log.log

# Parallel Stereo p2 (Step 8)

In [ ]:
!asap hirise step-eight {config} 2>&1 | tee -i -a ./8_parellel_stereo_2.log ./full_log.log

# Produce Previews (Step 9)

In [ ]:
!asap hirise step-nine 2>&1 | tee -i -a ./9_previews.log ./full_log.log

# Good Pixel Preview

In [ ]:
both = f'{left}_{right}'
img  = f'./{both}/results/{both}-GoodPixelMap.tif'
out  = img.replace('.tif', '.png')

In [ ]:
!gdal_translate -of PNG -co worldfile=yes {img} {out}

In [ ]:
Image(filename=out)

# Hillshade based initial transform to reference DEM

In [ ]:
!asap hirise pre-step-ten {ref_dem} --alignment-method {alignment_method} 2>&1 | tee -i -a ./10_pre_hillshade_align.log ./full_log.log

# Point Cloud align using initial transform from hillshade (Step 10)

In [ ]:
!asap hirise step-ten {max_disp} {ref_dem} --initial-transform hillshade_align/out-transform.txt 2>&1 | tee -i -a ./10_pc_align.log ./full_log.log

# Produce final aligned DEM, ortho, normalized image, error image (Step 11)

In [ ]:
!asap hirise step-eleven {demgsd} 2>&1 | tee -i -a ./11_dems_orthos.log ./full_log.log

# Adjust to Geoid (Step 12)

In [ ]:
!asap hirise step-twelve 2>&1 | tee -i -a ./12_geoid_adjustment.log ./full_log.log

# Make full-res Ortho image (Step 13)

In [ ]:
!asap hirise step-eleven {imggsd} --just-ortho True 2>&1 | tee -i -a ./13_img_full_ortho.log ./full_log.log